# VOLTANDO UMA PASTA PARA TRAZ

In [ ]:
import os
os.chdir("../")

# PIPELINE EXTRAÇÃO

In [ ]:
from modules.youtube.extract.ytbpy import channel, playlist, utils, video
from modules import db_manager
from datetime import datetime
from jinja2 import Template
from tqdm import tqdm
import pandas as pd
import yt_dlp
import time
import os

path_sql = "repository\\querys\\operation\\insert\\video.sql"
with open(path_sql, "r", encoding="utf-8") as file:
    sql_query_insert = file.read()
    
with open("repository\\querys\\operation\\read\\playlist.sql", "r", encoding="utf-8") as file:
    sql_query_read = file.read()
    df_playlist = db_manager.read_df(sql_query_read)
   
with open("repository\\querys\\operation\\read\\video.sql", "r", encoding="utf-8") as file:
    sql_query_read = file.read()
    df_videos = db_manager.read_df(sql_query_read)

In [ ]:
for ID_PLAYLIST, ID_GAME in tqdm(zip(df_playlist["id_playlist"], df_playlist["id_game"]), total=len(df_playlist), desc="Playlists"):
    time.sleep(1)
    videos = playlist.get_playlist_info(ID_PLAYLIST, max_results=100)
    if videos:
        videos = videos["videos"]
        for video in tqdm(videos, desc=f"Videos da playlist {ID_PLAYLIST}", leave=False):
            ID_VIDEO        = video["video_id"]
            cut = df_videos[df_videos.id_video.isin([ID_VIDEO])]
            if cut.empty:
                URL             = video["url"]
                TITLE           = video["title"]
                ID_PLAYLIST     = ID_PLAYLIST
                ID_GAME         = ID_GAME
                ID_PROPLAYER    = None
                ID_HERO         = None
                # TIME_MOVIE      = video["duration_seconds"] if "duration_seconds" in video else -1
                PUBLISHED_DATE  = video["published_date"] if "published_date" in video else "" 

                sql_template        = Template(sql_query_insert)
                rendered_sql        = sql_template.render(
                    ID_VIDEO        = ID_VIDEO,
                    URL             = URL,
                    ID_PLAYLIST     = ID_PLAYLIST,
                    ID_GAME         = ID_GAME,
                    ID_PROPLAYER    = ID_PROPLAYER,
                    ID_HERO         = ID_HERO,
                    # TIME_MOVIE      = int(TIME_MOVIE),
                    # PUBLISHED_DATE  = PUBLISHED_DATE,
                    REGISTER_DATE   = datetime.now().strftime("%Y-%m-%d")
                ).replace("'None'", "NULL").replace("None", "NULL")\
                .replace("'False'", "FALSE").replace("False", "FALSE")\
                .replace("'True'", "TRUE").replace("True", "TRUE")

                try:
                    db_manager.execute_raw(rendered_sql)
                except Exception as e:
                    print(f"Error inserting video data: {e}")
                    print(f"SQL: {rendered_sql}")
                    db_manager.connection.rollback()
                    time.sleep(120)  # Wait for 2 minutes before retrying
